In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

This notebook is a companion to the paper TBC, and reproduces the plots from the paper. We use the Ginibre point process as a running example. It is convenient, since its two-point correlation function and structure factor are analytically known.

In [ ]:
import numpy as np
from structure_factor.data import load_data
from structure_factor.structure_factor import StructureFactor
import structure_factor.utils as utils

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 25
mpl.rcParams['ytick.labelsize'] = 25
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=16)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# 1- Ginibre Ensemble 

In [ ]:
ginibre_pp = load_data.load_ginibre()

In [ ]:
from structure_factor.spatial_windows import BallWindow
window_show = BallWindow(center=(0,0), radius=50)
ginibre_pp.plot(window_res=window_show, 
                #file_name="ginibre_ball.png"
               )
                # we provide a method to plot the underlying sample

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp) 

In [ ]:
from structure_factor.spatial_windows import BoxWindow
L = ginibre_pp.window.radius/np.sqrt(2) # sidelength of the cubic window
print("Restricting the window to a cube of length", L)
bounds = np.array([[-L/2, L/2], [-L/2, L/2]]) 
window = BoxWindow(bounds) # create a cubic window
ginibre_pp_box = ginibre_pp.restrict_to_window(window) # create a Ginibre point pattern with box window
sf_ginibre_box = StructureFactor(ginibre_pp_box) # initialize a new instance of StructureFactor
#ginibre_pp_box.plot(file_name="ginibre_sample.png")

## 1.1- Direct spectral estimators (DSE)

### 1.1.1- Scattering intensity

In [ ]:

# on allowed values 
allowed_k, si = sf_ginibre_box.scattering_intensity(k_max=8)
print("Scattering intensity")
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre.plot_tapered_periodogram(allowed_k, si, axes=axis, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=80, error_bar=True, label=r"$\widehat{S}_{\mathrm{SI}}(\frac{2\pi\mathbf{n}}{L})$",
                                           #file_name="si_allowed_k.png"
                                          )

Scattering intensity on non allowed values and debiased scattering intensity

In [ ]:
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_ginibre_box.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_ginibre_box.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_ginibre_box.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre.plot_tapered_periodogram(k, si1, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=60, error_bar=True, label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$",
                                           file_name="si_k_ginibre.png"
                                          )

 all the above estimators in one picture

In [ ]:
norm_k_allowed = utils.norm_k(allowed_k)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', bins=60)
utils.plot_summary(norm_k, si1, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$", fmt='grey', bins=60 )
utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', bins=150 )
utils.plot_summary(norm_k, si3, axis=axis, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",fmt='c', bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

### 1.1.2- Tapered periodogram with sine taper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
s_tp = sf_ginibre_box.tapered_periodogram(k, taper, debiased=False)
s_dtp = sf_ginibre_box.tapered_periodogram(k, taper)
s_udtp = sf_ginibre_box.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre.plot_tapered_periodogram(k, s_dtp, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=100, error_bar=True, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$",
                                           file_name="s_dtp_sine_taper_ginibre.png"
                                          )

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_tp, axis=axis, label=r"$\widehat{S}_{\mathrm{TP}}(\mathbf{k})$", fmt='b', bins=60 )
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', bins=150 )
utils.plot_summary(norm_k, s_udtp, axis=axis, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",fmt='c', bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

 Result with scattering intensity

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='c', bins=150 )
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', bins=60)
utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{SIDTP}}(\mathbf{k})$", fmt='k', bins=150 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

sine taper looks better than scattering inetsnity

### 1.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
s_mtp = sf_ginibre_box.multitapered_periodogram(k, debiased=False)
s_mdtp = sf_ginibre_box.multitapered_periodogram(k)
s_mudtp = sf_ginibre_box.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre.plot_tapered_periodogram(k, s_mudtp, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=80, error_bar=True, label=r"$\widehat{S}_{\mathrm{MUDTP}}(\mathbf{k})$",
                                           #file_name="s_mudtp_sine_taper_ginibre.png"
                                          )

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_mtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MTP}}(\mathbf{k})$", fmt='b', ecolor="b", bins=60 )
utils.plot_summary(norm_k, s_mdtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MDTP}}(\mathbf{k})$", fmt='k', ecolor="k", bins=150 )
utils.plot_summary(norm_k, s_mudtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MUDTP}}(\mathbf{k})$",fmt='c', ecolor="c", bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

compaising with the best of the above

In [ ]:
norm_k = utils.norm_k(k)
norm_k_allowed = utils.norm_k(allowed_k)

fig, axis = plt.subplots(figsize=(10, 8))
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', ecolor='b', bins=50)
#utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{SIDTP}}(\mathbf{k})$", fmt='k', ecolor='k', bins=150 )
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', ecolor='k', bins=50 )
utils.plot_summary(norm_k, s_mdtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MDTP}}(\mathbf{k})$", fmt='m', ecolor="m", bins=50 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

## 1.2- Isotropic estimator (ISD)

### 1.2.1- Bartlett isotropic estimator

In [ ]:
k_norm, s_bi = sf_ginibre.bartlett_isotropic_estimator()

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre.plot_isotropic_estimator(k_norm, s_bi, exact_sf=utils.structure_factor_ginibre, axis=axis,
                                    label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_ginibre.png")

In [ ]:
k_non_allowed = np.linspace(min(k_norm) + 0.002, max(k_norm), 100)
k_norm_non_allowed, s_bi_non_allowed = sf_ginibre.bartlett_isotropic_estimator(k_non_allowed)
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis, exact_sf=utils.structure_factor_ginibre,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_ginibre_non_allowed.png")

### 1.2.2- Hankel transform estimation

#### 1.2.2.1- Pair correlation function

pcf.ppp

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

r= np.linspace(0, 10, 200)
ginibre_pcf_ppp = pcf.estimate(ginibre_pp, method="ppp", r=r, correction="all")

In [ ]:
pcf.plot(ginibre_pcf_ppp, exact_pcf=utils.pair_correlation_function_ginibre, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_ginibre.png")

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

ginibre_pcf_fv = pcf.estimate(ginibre_pp, method="fv", Kest=dict(rmax=30), fv=dict(method="b", spar=0.4))

In [ ]:
pcf.plot(ginibre_pcf_fv, exact_pcf=utils.pair_correlation_function_ginibre, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_ginibre.png")

Interpolation

In [ ]:
domain, ginibre_pcf_fct = pcf.interpolate(r=ginibre_pcf_fv["r"][1:], pcf_r=ginibre_pcf_fv["pcf"][1:])
r_max = domain["r_max"]
r_ = np.linspace(0, r_max, 200 )
plt.plot(r_, ginibre_pcf_fct(r_), 'b')
plt.plot(r_, utils.pair_correlation_function_ginibre(r_), 'g')

Ogata

In [ ]:
 # upper bound of the raduis on which the pcf has been approximated
k_norm = np.linspace(3,10, 300) # vector of wave length
k_norm, sf_Ogata = sf_ginibre.hankel_quadrature(ginibre_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.1,
                                                    nb_points=1000, r_max=r_max)

In [ ]:
fig = sf_ginibre.plot_isotropic_estimator(k_norm, sf_Ogata, 
                                           exact_sf=utils.structure_factor_ginibre,
                                          label=r"$\widehat{S}_{\mathrm{HO}}(k)$",
                                            #bins=60,
                                            #error_bar=True,
                                            file_name="ginibre_s_ho.png"
                                           )


Baddour-Chouinard

In [ ]:
k = np.linspace(0.5, 10, 200)
k_norm, s_hbc = sf_ginibre.hankel_quadrature(ginibre_pcf_fct,k_norm=k,  r_max=r_max,  nb_points=1000 )

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_ginibre.plot_isotropic_estimator(k_norm, s_hbc, axis=axis, exact_sf=utils.structure_factor_ginibre,
                                    label=r"$\widehat{S}_{\mathrm{HBC}}(k)$",file_name="ginibre_s_hbc.png")

# 2- Poisson point process

In this section we repeat the previous experience on a point configuration from a standard homogeneous Poisson Point Process in a cubic window

In [ ]:
from structure_factor.data import load_data
poisson_pp = load_data.load_poisson()
poisson_pp_res = poisson_pp.restrict_to_window(window)
sf_poisson_res = StructureFactor(poisson_pp_res) # initializing the StructureFactor class

In [ ]:
poisson_pp_res.plot()

## 2.1- Direct sprectral estimator

### 2.1.2- Scattering intensity

In [ ]:

# on allowed values 
allowed_k, si = sf_poisson_res.scattering_intensity(k_max=5)
print("Scattering intensity")
k_norm = np.linalg.norm(allowed_k, axis=1)
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_res.plot_tapered_periodogram(allowed_k, si, axes=axis, plot_type="radial",exact_sf=utils.structure_factor_poisson,
                                               label=r"$\widehat{S}_{\mathrm{SI}}(\frac{2\pi\mathbf{n}}{L})$",
                                            bins=40, error_bar=True, file_name="si_allowed_k_pois.png")

In [ ]:
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_poisson_res.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_poisson_res.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_poisson_res.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_res.plot_scattering_intensity(k, si3, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_poisson,
                                               label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",
                                            bins=65, error_bar=True, file_name="si_unddebiased_pois.png")

### 2.1.2- Tapered periodogram with SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
s_tp = sf_poisson_res.tapered_periodogram(k, taper, debiased=False)
s_dtp = sf_poisson_res.tapered_periodogram(k, taper)
s_udtp = sf_poisson_res.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_res.plot_scattering_intensity(k, s_udtp, axes=axis, positive=False, plot_type="radial",
                                            bins=100, error_bar=True, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",
                                           file_name="s_udtp_sine_taper_poisson.png"
                                          )

### 2.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
s_mtp = sf_poisson_res.multitapered_periodogram(k, debiased=False)
s_mdtp = sf_poisson_res.multitapered_periodogram(k, P=2)
s_mudtp = sf_poisson_res.multitapered_periodogram(k, debiased=True, direct=False, P=2)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_res.plot_tapered_periodogram(k, s_mudtp, axes=axis, positive=False, plot_type="radial",
                                            bins=60, error_bar=True, label=r"$\widehat{S}_{\mathrm{MUDTP}}(\mathbf{k})$",
                                           file_name="s_mudtp_sine_taper_poisson.png"
                                          )

## 2.2- Isotropic estimators

In [ ]:
from structure_factor.spatial_windows import BallWindow
from structure_factor.point_pattern import PointPattern
from structure_factor.homogeneous_poisson_process import HomogeneousPoissonPointProcess
poisson_ = HomogeneousPoissonPointProcess(intensity=1)
window_show = BallWindow(center=(0,0), radius=100)
poisson_points_ball = poisson_.generate_sample(window=window_show)
poisson_pp_ball = PointPattern(poisson_points_ball, window=window_show, intensity=1)
poisson_pp_ball.plot( file_name="poisson_ball.png")

In [ ]:
sf_poisson_ball = StructureFactor(poisson_pp_ball)

### 2.2.1- Bartlett isotropic estimators

In [ ]:
k_norm, s_bi = sf_poisson_ball.bartlett_isotropic_estimator()
fig, axis = plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson.png")

In [ ]:
sf_poisson_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson.png")

In [ ]:
k_non_allowed = np.linspace(min(k_norm)+0.002, max(k_norm) + 0.002, 100)
k_norm_non_allowed, s_bi_non_allowed = sf_poisson_ball.bartlett_isotropic_estimator(k_non_allowed)
fig, axis = plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson_non_allowed.png")

In [ ]:
sf_poisson_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson_non_allowed.png")
plt.show()

### 2.2.2- Pair correlation function

pcf.ppp

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

r= np.linspace(0, 10, 200)
poisson_pcf_ppp = pcf.estimate(poisson_pp_ball, method="ppp", r=r, correction="all")
pcf.plot(poisson_pcf_ppp, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_poisson.png")

pcf.fv

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

poisson_pcf_fv = pcf.estimate(poisson_pp_ball, method="fv", Kest=dict(rmax=30), fv=dict(method="b", spar=1))
pcf.plot(poisson_pcf_fv, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_poisson.png")

Interpolation

In [ ]:
domain, poisson_pcf_fct = pcf.interpolate(r=poisson_pcf_fv["r"][1:], pcf_r=poisson_pcf_fv["pcf"][1:])

r_max = domain["r_max"] # upper bound of the raduis on which the pcf has been approximated

Ogata

In [ ]:

k_norm = np.linspace(0.4,10, 3000) # vector of wave length
k_norm, sf_Ogata = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.1,
                                                    nb_points=1000, r_max=r_max)

fig = sf_thomas_ball.plot_isotropic_estimator(k_norm, sf_Ogata, 
                                           exact_sf=sf_th_exact,
                                            #bins=60,
                                            #error_bar=True,
                                            file_name="thomas_s_ho.png"
                                           )

Baddour-Chouinard

In [ ]:
k = np.linspace(0.5, 10, 200)
k_norm, s_hbc = sf_poisson_ball.hankel_quadrature(poisson_pcf_fct,
                                                  k_norm=k,
                                                   r_max=r_max, 
                                                   nb_points=1000
                                                   )
fig, axis= plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm, s_hbc, axis=axis, label=r"$\widehat{S}_{\mathrm{HBC}}$", file_name="poisson_s_hbc.png"
                                           )

# 3- KLY process

In [ ]:
kly_pp = load_data.load_kly()
L1 = kly_pp.window.bounds[1][1]
kly_pp.points = kly_pp.points - L1/2
kly_pp.window = BoxWindow([[-L1/2, L1/2], [-L1/2, L1/2]])
kly_pp_res = kly_pp.restrict_to_window(window)
kly_pp_res.plot()

## 3.1- Direct spectral estimator

### 3.1.1- Scattering intensity

In [ ]:
sf_kly_res = StructureFactor(kly_pp_res)

In [ ]:

# on allowed values 
params={"k_max",2}
allowed_k, si = sf_kly_res.scattering_intensity(k_max=15)
print("Scattering intensity")



In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_res.plot_tapered_periodogram(allowed_k, si, axes=axis, plot_type="radial",
                                           label=r"$\widehat{S}_{\mathrm{SI}}(\frac{2\pi\mathbf{n}}{L})$",
                                            bins=80, error_bar=True, 
                                           #file_name="si_allowed_k_kly.png"
                                          )

In [ ]:
x = np.linspace(-15, 15, 340)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_kly_res.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_kly_res.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_kly_res.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_res.plot_tapered_periodogram(k, si1, axes=axis, positive=False, plot_type="radial",
                                           label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$",
                                            bins=60, error_bar=True, file_name="si_k_kly.png")

### 3.1.2- Tapered periodogram with SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
s_tp = sf_kly_res.tapered_periodogram(k, taper, debiased=False)
s_dtp = sf_kly_res.tapered_periodogram(k, taper)
s_udtp = sf_kly_res.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_res.tapered_periodogram(k, s_udtp, axes=axis, positive=False, plot_type="radial",
                                            bins=100, error_bar=True, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",
                                           file_name="s_udtp_sine_taper_kly.png"
                                          )

### 3.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
s_mtp = sf_kly_res.multitapered_periodogram(k, debiased=False)
s_mdtp = sf_kly_res.multitapered_periodogram(k)
s_mudtp = sf_kly_res.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_res.plot_tapered_periodogram(k, s_mtp, axes=axis, positive=False, plot_type="radial",
                                            bins=80, error_bar=True, label=r"$\widehat{S}_{\mathrm{MTP}}(\mathbf{k})$",
                                            file_name="s_mtp_sine_taper_kly.png"
                                          )

## 3.2- Isotropic estimators

In [ ]:
kly_pp_ball = kly_pp.restrict_to_window(BallWindow(center=(0,0), radius=100))
window_show = BallWindow(center=(0,0), radius=50)
#kly_pp_ball.plot(window_res=window_show, file_name="kly_ball.png")

In [ ]:
sf_kly_ball = StructureFactor(kly_pp_ball)

### 3.2.1- Bartlett isotropic estimators

In [ ]:

k_norm, s_bi = sf_kly_ball.bartlett_isotropic_estimator(n_allowed_k_norm=200)
fig, axis = plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_kly.png")

In [ ]:
k_non_allowed = np.linspace(min(k_norm)+ 0.003, max(k_norm), 200)
k_norm_non_allowed, s_bi_non_allowed = sf_kly_ball.bartlett_isotropic_estimator(k_non_allowed)
fig, axis = plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_kly_non_allowed.png")

### 3.2.2- Pair correlation function

pcf.ppp

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

r= np.linspace(0, 10, 200)
kly_pcf_ppp = pcf.estimate(kly_pp_ball, method="ppp", r=r, correction="all")
pcf.plot(kly_pcf_ppp, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_kly.png")

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

kly_pcf_fv = pcf.estimate(kly_pp_ball, method="fv", Kest=dict(rmax=30), fv=dict(method="b", spar=0.4))
pcf.plot(kly_pcf_fv, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_kly.png")

Interpolation

In [ ]:
domain, kly_pcf_fct = pcf.interpolate(r=kly_pcf_fv["r"][1:], pcf_r=kly_pcf_fv["pcf"][1:])
r_max = domain["r_max"] # upper bound of the raduis on which the pcf has been approximated


### 3.2.3- Hankel transform

Ogata

In [ ]:
k_norm = np.linspace(0.4,10, 3000) # vector of wave length
k_norm, sf_Ogata = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.1,
                                                    nb_points=1000, r_max=r_max)


In [ ]:

fig = sf_thomas_ball.plot_isotropic_estimator(k_norm, sf_Ogata, 
                                           exact_sf=sf_th_exact,
                                            #bins=60,
                                            #error_bar=True,
                                            file_name="thomas_s_ho.png"
                                           )

Baddour-Chouinard

In [ ]:
k = np.linspace(0.5, 10, 200)
k_norm, s_hbc = sf_kly_ball.hankel_quadrature(kly_pcf_fct, k_norm=k, r_max=r_max, nb_points=1000)

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm, s_hbc, axis=axis, label=r"$\widehat{S}_{\mathrm{HBC}}(k)$" , file_name="kly_s_hbc.png")

# 4- Thomas process

In [ ]:

L= 200
# Simulation window parameters
xMin = -L/2;
xMax = L/2;
yMin = -L/2;
yMax = L/2;

# Parameters for the parent and daughter point processes
lambdaParent = 1/(10*np.pi);  # density of parent Poisson point process
lambdaDaughter = 10*np.pi;  # mean number of points in each cluster
sigma = 2;  # sigma for normal variables (ie random locations) of daughters

# Extended simulation windows parameters
rExt=6*sigma; # extension parameter 
# for rExt, use factor of deviation sigma eg 5 or 6
xMinExt = xMin - rExt;
xMaxExt = xMax + rExt;
yMinExt = yMin - rExt;
yMaxExt = yMax + rExt;
# rectangle dimensions
xDeltaExt = xMaxExt - xMinExt;
yDeltaExt = yMaxExt - yMinExt;
areaTotalExt = xDeltaExt * yDeltaExt;  # area of extended rectangle

# Simulate Poisson point process for the parents
numbPointsParent = np.random.poisson(areaTotalExt * lambdaParent);# Poisson number of points
# x and y coordinates of Poisson points for the parent
xxParent = xMinExt + xDeltaExt * np.random.uniform(0, 1, numbPointsParent);
yyParent = yMinExt + yDeltaExt * np.random.uniform(0, 1, numbPointsParent);

# Simulate Poisson point process for the daughters (ie final poiint process)
numbPointsDaughter = np.random.poisson(lambdaDaughter, numbPointsParent);
numbPoints = sum(numbPointsDaughter);  # total number of points

# Generate the (relative) locations in Cartesian coordinates by
# simulating independent normal variables
xx0 = np.random.normal(0, sigma, numbPoints);  # (relative) x coordinaets
yy0 = np.random.normal(0, sigma, numbPoints);  # (relative) y coordinates

# replicate parent points (ie centres of disks/clusters)
xx = np.repeat(xxParent, numbPointsDaughter);
yy = np.repeat(yyParent, numbPointsDaughter);

# translate points (ie parents points are the centres of cluster disks)
xx = xx + xx0;
yy = yy + yy0;

# thin points if outside the simulation window
booleInside = ((xx >= xMin) & (xx <= xMax) & (yy >= yMin) & (yy <= yMax));
# retain points inside simulation window
xx = xx[booleInside];  
yy = yy[booleInside];

# Plotting
plt.scatter(xx, yy, edgecolor='b', facecolor='none', alpha=0.5, s=2);
plt.xlabel("x");
plt.ylabel("y");
plt.axis('equal');

In [ ]:
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BoxWindow, BallWindow

thomas_points = np.column_stack((xx , yy))
bounds = [[xMin, xMax], [yMin, yMax]]
thomas_window = BoxWindow(bounds)
thomas_intensity = lambdaParent*lambdaDaughter
intensity_parent = lambdaParent
sigma = sigma
thomas_pp = PointPattern(thomas_points, thomas_window )
#thomas_pp.plot(file_name="thomas_pp.png")

In [ ]:
sf_thomas_res = StructureFactor(thomas_pp)
sf_th_exact = lambda k: utils.structure_factor_thomas(k, lambdaDaughter, sigma )

## 4.1- Direct spectral estimator

### 4.1.1- Scattering intensity

In [ ]:

# on allowed values 
allowed_k, si = sf_thomas_res.scattering_intensity(k_max=6)
print("Scattering intensity")



In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_res.plot_scattering_intensity(allowed_k, si, axes=axis, plot_type="radial",exact_sf=sf_th_exact,
                                            bins=80, error_bar=True, 
                                            label=r"$\widehat{S}_{\mathrm{SI}}(\frac{2\pi\mathbf{n}}{L})$",
                                            file_name="si_allowed_k_thomas.png"
                                             )

In [ ]:
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_thomas_res.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_thomas_res.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_thomas_res.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_res.plot_scattering_intensity(k, si1, axes=axis, plot_type="radial",exact_sf=sf_th_exact,
                                            bins=60, error_bar=True, 
                                            label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$",
                                            file_name="si_k_thomas.png"
                                             )

### 4.1.2- Tapered periodogram SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
s_tp = sf_thomas_res.tapered_periodogram(k, taper, debiased=False)
s_dtp = sf_thomas_res.tapered_periodogram(k, taper)
s_udtp = sf_thomas_res.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_res.plot_tapered_periodogram(k, s_dtp, axes=axis, positive=False, plot_type="radial", exact_sf=sf_th_exact,
                                            bins=80, error_bar=True, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$",
                                           #file_name="s_tp_sine_taper_thomas.png"
                                          )

### 4.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(-8, 8, 300)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
s_mtp = sf_thomas_res.multitapered_periodogram(k, debiased=False)
s_mdtp = sf_thomas_res.multitapered_periodogram(k, debiased=True, direct=True) #or simply sf_thomas_res.multitapered_periodogram(k)
s_mudtp = sf_thomas_res.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_res.plot_tapered_periodogram(k, s_mtp, axes=axis, positive=False, plot_type="radial",exact_sf=sf_th_exact,
                                            bins=80, error_bar=True, label=r"$\widehat{S}_{\mathrm{MTP}}(\mathbf{k})$",
                                            #file_name="s_mtp_sine_taper_thomas.png"
                                          )

## 4.2- Isotropic estimators

In [ ]:
from structure_factor.point_pattern import PointPattern

thomas_points = np.column_stack((xx , yy))
bounds = [[-200, 200], [-200, 200]]
thomas_window = BoxWindow(bounds)
thomas_intensity = lambdaParent*lambdaDaughter
intensity_parent = lambdaParent
sigma = sigma
thomas_pp = PointPattern(thomas_points, thomas_window )

In [ ]:
thomas_pp_ball = thomas_pp.restrict_to_window(BallWindow(center=(0,0), radius=100))
window_show = BallWindow(center=(0,0), radius=50)
thomas_pp_ball.plot( file_name="thomas_ball.png")

In [ ]:
sf_thomas_ball = StructureFactor(thomas_pp_ball)

### 4.2.1- Bartlett isotropic estimator

In [ ]:

k_norm, s_bi = sf_thomas_ball.bartlett_isotropic_estimator(n_allowed_k_norm=150)


In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_th_exact = lambda k: utils.structure_factor_thomas(k, lambdaDaughter, sigma )
sf_thomas_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis, exact_sf=sf_th_exact,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_thomas.png")

In [ ]:
k = np.linspace(min(k_norm) + 0.002, max(k_norm), 150)
k_norm_non_allowed, s_bi_nonallowed = sf_thomas_ball.bartlett_isotropic_estimator(k_norm=k)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_th_exact = lambda k: utils.structure_factor_thomas(k, lambdaDaughter, sigma )
sf_thomas_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_nonallowed, axis=axis, exact_sf=sf_th_exact,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_thomas_non_allowed_k.png")

### 4.2.2- Pair correlation function

pcf.ppp

In [ ]:
ext_pcf_thomas = lambda r: utils.pair_correlation_function_thomas(r, d=2, rho_parent=lambdaParent, sigma=sigma)

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

r= np.linspace(0, 10, 200)
thomas_pcf_ppp = pcf.estimate(thomas_pp_ball, method="ppp", r=r, correction="all")
pcf.plot(thomas_pcf_ppp, figsize=(7,6),
         exact_pcf=ext_pcf_thomas,
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_thomas.png")

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

thomas_pcf_fv = pcf.estimate(thomas_pp_ball, method="fv", Kest=dict(rmax=30), fv=dict(method="b", spar=0.3))


In [ ]:
thomas_pcf_fv

In [ ]:
pcf.plot(thomas_pcf_fv, figsize=(7,6),
         exact_pcf=ext_pcf_thomas,
         color=['grey'], style=["."],
         #file_name="pcf_fv_thomas.png"
        )

Interpolation

In [ ]:
domain, thomas_pcf_fct = pcf.interpolate(r=thomas_pcf_fv["r"][1:], pcf_r=thomas_pcf_fv["pcf"][1:])
r_max = domain["r_max"]

In [ ]:
r_ = np.linspace(0, 20, 200 )
plt.plot(r, thomas_pcf_fct(r), 'b')
plt.plot(r, ext_pcf_thomas(r), 'g')

#### 4.2.3- Hankel transform 

Ogata

In [ ]:
r_max = domain["r_max"] # upper bound of the raduis on which the pcf has been approximated
k_norm = np.linspace(0.4,10, 3000) # vector of wave length
k_norm, sf_Ogata = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.1,
                                                    nb_points=1000, r_max=r_max)


In [ ]:
fig = sf_thomas_ball.plot_isotropic_estimator(k_norm, sf_Ogata, 
                                           exact_sf=sf_th_exact,
                                            #bins=60,
                                            #error_bar=True,
                                            file_name="thomas_s_ho.png"
                                           )

Baddour chouinard

In [ ]:
k = np.linspace(0.1, 10, 200)
k_norm, s_hbc = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, k_norm=k,  r_max=r_max, nb_points=1000)

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_thomas_ball.plot_isotropic_estimator(k_norm, s_hbc,axis=axis, exact_sf=sf_th_exact,label= r"$\widehat{S}_{\mathrm{HBC}}(k)$",file_name="thomas_s_hbc.png" )

## Sine taper

In [ ]:
from structure_factor.tapers import SineTaper

In [ ]:
widths = np.diff(window_.bounds.T, axis=0)
widths.shape

In [ ]:
x = np.linspace(-10, 10, 100)
x = np.array([x]).T
p = 6
sin_t = SineTaper(p)
window_ = BoxWindow([-5, 5])
tp_x = sin_t.taper(x, window_)

In [ ]:
plt.plot(x, tp_x, 'b.')

In [ ]:
x.shape

# Section 2

In [ ]:
ax = ginibre_pp_box.plot()

In [ ]:
th_pp = Poi